In [2]:
# Import libraries
import numpy as np
import pandas as pd
import pickle
import joblib
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries loaded!")

✓ Libraries loaded!


In [3]:
# Import libraries
import numpy as np
import pandas as pd
import pickle
import joblib
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries loaded!")

✓ Libraries loaded!


In [4]:
# Load all required data and models
import os
os.chdir(r'c:\Users\gaspa\Desktop\machine learning\projet')

# Load synergy and counter data
with open('data/synergies.pkl', 'rb') as f:
    synergy_data = pickle.load(f)
    
with open('data/counters.pkl', 'rb') as f:
    counter_data = pickle.load(f)

# Load the best model (Stacking Enhanced or XGBoost as fallback)
try:
    best_model = joblib.load('models/stacking_enhanced.pkl')
    print("✓ Loaded Stacking Enhanced model")
except:
    best_model = joblib.load('models/xgboost_enhanced.pkl')
    print("✓ Loaded XGBoost Enhanced model")

# Load feature names
with open('data/enhanced_feature_names.pkl', 'rb') as f:
    feature_names = pickle.load(f)

# Load original data for champion list
df = pd.read_csv('data/s11.csv')
all_champions = set()
for col in ['team_1__001', 'team_1__002', 'team_1__003', 'team_1__004', 'team_1__005',
            'team_2__001', 'team_2__002', 'team_2__003', 'team_2__004', 'team_2__005']:
    all_champions.update(df[col].unique())
all_champions = sorted(list(all_champions))

print(f"✓ Data loaded!")
print(f"  - {len(all_champions)} champions available")
print(f"  - {len(synergy_data)} synergy pairs")
print(f"  - {len(counter_data)} counter matchups")

✓ Loaded Stacking Enhanced model
✓ Data loaded!
  - 153 champions available
  - 2597 synergy pairs
  - 5958 counter matchups


In [5]:
# Champion roles database
CHAMPION_ROLES = {
    # ADC
    'Jinx': ['ADC'], 'Caitlyn': ['ADC'], 'Ezreal': ['ADC'], 'Vayne': ['ADC'], "Kai'Sa": ['ADC'],
    'Jhin': ['ADC'], 'Miss Fortune': ['ADC'], 'Ashe': ['ADC'], 'Draven': ['ADC'], 'Lucian': ['ADC', 'MID'],
    'Tristana': ['ADC', 'MID'], 'Samira': ['ADC'], 'Aphelios': ['ADC'], 'Xayah': ['ADC'], 'Sivir': ['ADC'],
    'Twitch': ['ADC'], "Kog'Maw": ['ADC'], 'Varus': ['ADC'], 'Kalista': ['ADC'], 'Senna': ['ADC', 'SUPPORT'],
    
    # SUPPORT
    'Thresh': ['SUPPORT'], 'Leona': ['SUPPORT'], 'Nautilus': ['SUPPORT'], 'Lulu': ['SUPPORT'],
    'Nami': ['SUPPORT'], 'Janna': ['SUPPORT'], 'Soraka': ['SUPPORT'], 'Yuumi': ['SUPPORT'],
    'Blitzcrank': ['SUPPORT'], 'Morgana': ['SUPPORT', 'MID'], 'Alistar': ['SUPPORT'], 'Braum': ['SUPPORT'],
    'Rakan': ['SUPPORT'], 'Pyke': ['SUPPORT'], 'Bard': ['SUPPORT'], 'Zyra': ['SUPPORT'],
    'Seraphine': ['SUPPORT', 'MID'], 'Lux': ['SUPPORT', 'MID'], 'Karma': ['SUPPORT'],
    
    # MID
    'Ahri': ['MID'], 'Zed': ['MID'], 'Yasuo': ['MID', 'TOP'], 'Yone': ['MID', 'TOP'],
    'Syndra': ['MID'], 'Orianna': ['MID'], 'Viktor': ['MID'], 'Azir': ['MID'],
    'Katarina': ['MID'], 'Akali': ['MID', 'TOP'], 'LeBlanc': ['MID'], 'Fizz': ['MID'],
    'Sylas': ['MID', 'TOP'], 'Veigar': ['MID'], 'Annie': ['MID'], 'Twisted Fate': ['MID'],
    'Kassadin': ['MID'], 'Galio': ['MID', 'SUPPORT'], 'Talon': ['MID'], 'Ekko': ['MID', 'JUNGLE'],
    'Diana': ['MID', 'JUNGLE'], 'Zoe': ['MID'], 'Anivia': ['MID'], 'Malzahar': ['MID'],
    'Xerath': ['MID'], "Vel'Koz": ['MID'], 'Ziggs': ['MID'], 'Corki': ['MID'],
    
    # JUNGLE
    'Lee Sin': ['JUNGLE'], "Kha'Zix": ['JUNGLE'], 'Graves': ['JUNGLE'], 'Nidalee': ['JUNGLE'],
    'Elise': ['JUNGLE'], "Rek'Sai": ['JUNGLE'], 'Hecarim': ['JUNGLE'], 'Olaf': ['JUNGLE', 'TOP'],
    'Jarvan IV': ['JUNGLE'], 'Vi': ['JUNGLE'], 'Sejuani': ['JUNGLE'], 'Zac': ['JUNGLE'],
    'Nunu & Willump': ['JUNGLE'], 'Ivern': ['JUNGLE'], 'Kindred': ['JUNGLE'], 'Lillia': ['JUNGLE'],
    'Viego': ['JUNGLE'], 'Kayn': ['JUNGLE'], 'Rengar': ['JUNGLE'], 'Evelynn': ['JUNGLE'],
    'Shaco': ['JUNGLE'], 'Fiddlesticks': ['JUNGLE'], 'Amumu': ['JUNGLE'], 'Rammus': ['JUNGLE'],
    'Warwick': ['JUNGLE'], 'Udyr': ['JUNGLE'], 'Volibear': ['JUNGLE', 'TOP'], 'Skarner': ['JUNGLE'],
    'Nocturne': ['JUNGLE'], 'Xin Zhao': ['JUNGLE'], 'Taliyah': ['JUNGLE', 'MID'],
    
    # TOP
    'Darius': ['TOP'], 'Garen': ['TOP'], 'Fiora': ['TOP'], 'Camille': ['TOP'],
    'Riven': ['TOP'], 'Irelia': ['TOP', 'MID'], 'Jax': ['TOP'], 'Renekton': ['TOP'],
    'Aatrox': ['TOP'], 'Sett': ['TOP', 'SUPPORT'], 'Mordekaiser': ['TOP'], 'Urgot': ['TOP'],
    'Ornn': ['TOP'], 'Malphite': ['TOP'], 'Maokai': ['TOP', 'SUPPORT'], 'Shen': ['TOP'],
    'Poppy': ['TOP', 'JUNGLE'], 'Gnar': ['TOP'], 'Kennen': ['TOP'], 'Jayce': ['TOP', 'MID'],
    'Gangplank': ['TOP'], 'Nasus': ['TOP'], 'Illaoi': ['TOP'], 'Kled': ['TOP'],
    'Teemo': ['TOP'], 'Quinn': ['TOP'], 'Kayle': ['TOP'], "Cho'Gath": ['TOP'],
    'Dr. Mundo': ['TOP'], 'Singed': ['TOP'], 'Tryndamere': ['TOP'], 'Wukong': ['TOP', 'JUNGLE'],
    'Pantheon': ['TOP', 'MID', 'SUPPORT'], 'Gragas': ['TOP', 'JUNGLE'],
}

def get_champion_role(champion):
    """Get primary role for a champion"""
    return CHAMPION_ROLES.get(champion, ['FLEX'])[0]

print(f"✓ Role database loaded with {len(CHAMPION_ROLES)} champions")

✓ Role database loaded with 132 champions


In [6]:
def calculate_team_synergy(team_champions):
    """Calculate total synergy score for a team"""
    if len(team_champions) < 2:
        return 0.0
    
    total_synergy = 0.0
    
    for c1, c2 in combinations(team_champions, 2):
        pair = tuple(sorted([c1, c2]))
        if pair in synergy_data:
            # Extract synergy_score from the dict
            score = synergy_data[pair]
            if isinstance(score, dict):
                total_synergy += score.get('synergy_score', 0)
            else:
                total_synergy += score
    
    return total_synergy

def calculate_counter_score(team_champions, enemy_champions):
    """Calculate counter advantage score"""
    total_counter = 0.0
    
    for team_champ in team_champions:
        for enemy_champ in enemy_champions:
            matchup = (team_champ, enemy_champ)
            if matchup in counter_data:
                # Extract counter_score from the dict
                score = counter_data[matchup]
                if isinstance(score, dict):
                    total_counter += score.get('counter_score', 0)
                else:
                    total_counter += score
    
    return total_counter

def get_role_coverage(champions):
    """Calculate role coverage for a team"""
    roles_covered = set()
    for champ in champions:
        if champ in CHAMPION_ROLES:
            roles_covered.update(CHAMPION_ROLES[champ])
    return len(roles_covered.intersection({'TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT'})) / 5

print("✓ Helper functions defined!")

✓ Helper functions defined!


In [7]:
def create_features_for_prediction(team_champions, enemy_champions):
    """Create feature vector for a draft state"""
    features = {}
    
    # One-hot encoding for champions
    for champ in all_champions:
        features[f'team_{champ}'] = 1 if champ in team_champions else 0
        features[f'enemy_{champ}'] = 1 if champ in enemy_champions else 0
    
    # Role features
    role_counts = {'TOP': 0, 'JUNGLE': 0, 'MID': 0, 'ADC': 0, 'SUPPORT': 0}
    enemy_role_counts = {'TOP': 0, 'JUNGLE': 0, 'MID': 0, 'ADC': 0, 'SUPPORT': 0}
    
    for champ in team_champions:
        role = get_champion_role(champ)
        if role in role_counts:
            role_counts[role] += 1
    
    for champ in enemy_champions:
        role = get_champion_role(champ)
        if role in enemy_role_counts:
            enemy_role_counts[role] += 1
    
    for role in ['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT']:
        features[f'team_{role.lower()}_count'] = role_counts[role]
        features[f'enemy_{role.lower()}_count'] = enemy_role_counts[role]
    
    # Role coverage
    features['team_role_coverage'] = get_role_coverage(team_champions)
    features['enemy_role_coverage'] = get_role_coverage(enemy_champions)
    features['role_coverage_diff'] = features['team_role_coverage'] - features['enemy_role_coverage']
    
    # Synergy features
    team_synergy = calculate_team_synergy(team_champions)
    enemy_synergy = calculate_team_synergy(enemy_champions)
    
    features['team_synergy_total'] = team_synergy
    features['team_synergy_avg'] = team_synergy / max(1, len(list(combinations(team_champions, 2))))
    features['enemy_synergy_total'] = enemy_synergy
    features['enemy_synergy_avg'] = enemy_synergy / max(1, len(list(combinations(enemy_champions, 2))))
    features['synergy_advantage'] = team_synergy - enemy_synergy
    
    # Counter features
    counter_score = calculate_counter_score(team_champions, enemy_champions)
    matchups = len(team_champions) * len(enemy_champions)
    
    features['counter_advantage_total'] = counter_score
    features['counter_advantage_avg'] = counter_score / max(1, matchups)
    
    # Create feature vector in correct order
    feature_vector = []
    for fname in feature_names:
        feature_vector.append(features.get(fname, 0))
    
    return np.array(feature_vector).reshape(1, -1)

print("✓ Feature creation function ready!")

✓ Feature creation function ready!


In [8]:
def recommend_champions(team_champions, enemy_champions, n_recommendations=10, role_filter=None):
    """
    Recommend best champions to pick given current draft state.
    
    Args:
        team_champions: List of champions already on your team
        enemy_champions: List of champions on enemy team
        n_recommendations: Number of recommendations to return
        role_filter: Filter by role ('TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT')
    
    Returns:
        DataFrame with champion recommendations and win probabilities
    """
    # Get available champions (not already picked)
    picked = set(team_champions + enemy_champions)
    available = [c for c in all_champions if c not in picked]
    
    # Filter by role if specified
    if role_filter:
        available = [c for c in available if c in CHAMPION_ROLES and role_filter in CHAMPION_ROLES[c]]
    
    recommendations = []
    
    for champion in available:
        # Create hypothetical team with this champion
        hypothetical_team = team_champions + [champion]
        
        # Get features and predict
        features = create_features_for_prediction(hypothetical_team, enemy_champions)
        
        try:
            win_prob = best_model.predict_proba(features)[0][1]
        except:
            win_prob = 0.5
        
        # Calculate synergy with current team
        synergy_gain = 0
        for teammate in team_champions:
            pair = tuple(sorted([champion, teammate]))
            if pair in synergy_data:
                score = synergy_data[pair]
                if isinstance(score, dict):
                    synergy_gain += score.get('synergy_score', 0)
                else:
                    synergy_gain += score
        
        # Calculate counter advantage vs enemies
        counter_gain = 0
        for enemy in enemy_champions:
            matchup = (champion, enemy)
            if matchup in counter_data:
                score = counter_data[matchup]
                if isinstance(score, dict):
                    counter_gain += score.get('counter_score', 0)
                else:
                    counter_gain += score
        
        recommendations.append({
            'Champion': champion,
            'Role': get_champion_role(champion),
            'Win %': f"{win_prob*100:.1f}%",
            'Win Prob': win_prob,
            'Synergy': f"{synergy_gain:+.2f}",
            'Counter': f"{counter_gain:+.2f}"
        })
    
    # Sort by win probability
    recommendations = sorted(recommendations, key=lambda x: x['Win Prob'], reverse=True)
    
    # Create DataFrame
    df_rec = pd.DataFrame(recommendations[:n_recommendations])
    df_rec = df_rec[['Champion', 'Role', 'Win %', 'Synergy', 'Counter']]
    
    return df_rec

print("✓ Recommendation function ready!")

✓ Recommendation function ready!


---
## 🎯 Interactive Draft Recommendation

Use the functions below to get champion recommendations during draft!

In [9]:
# ========================================
# 📝 EXAMPLE 1: Early Draft (Pick 1-2)
# ========================================

# Your team has picked:
my_team = ['Leona']  # Support locked in

# Enemy team has picked:
enemy_team = ['Thresh', 'Lee Sin']  # They have support and jungle

print("="*60)
print("🎮 DRAFT SITUATION")
print("="*60)
print(f"\n📘 Your Team: {my_team if my_team else 'None yet'}")
print(f"📕 Enemy Team: {enemy_team if enemy_team else 'None yet'}")
print("\n" + "="*60)
print("🏆 TOP 10 RECOMMENDED PICKS")
print("="*60 + "\n")

recommendations = recommend_champions(my_team, enemy_team, n_recommendations=10)
print(recommendations.to_string(index=False))

🎮 DRAFT SITUATION

📘 Your Team: ['Leona']
📕 Enemy Team: ['Thresh', 'Lee Sin']

🏆 TOP 10 RECOMMENDED PICKS

    Champion   Role Win % Synergy Counter
Fiddlesticks JUNGLE 85.4%   +0.33   +0.00
    Katarina    MID 85.0%   +0.25   -0.30
       Kayle    TOP 84.8%   +0.25   -0.17
      Singed    TOP 81.6%   +0.10   +0.00
      Rengar JUNGLE 81.5%   +0.10   -0.32
       Riven    TOP 81.3%   +0.12   -0.17
    Kassadin    MID 80.5%   +0.17   -0.25
      Darius    TOP 80.5%   +0.17   -0.50
     Kha'Zix JUNGLE 80.3%   +0.10   -0.50
    Tristana    ADC 80.3%   +0.33   -0.83
    Champion   Role Win % Synergy Counter
Fiddlesticks JUNGLE 85.4%   +0.33   +0.00
    Katarina    MID 85.0%   +0.25   -0.30
       Kayle    TOP 84.8%   +0.25   -0.17
      Singed    TOP 81.6%   +0.10   +0.00
      Rengar JUNGLE 81.5%   +0.10   -0.32
       Riven    TOP 81.3%   +0.12   -0.17
    Kassadin    MID 80.5%   +0.17   -0.25
      Darius    TOP 80.5%   +0.17   -0.50
     Kha'Zix JUNGLE 80.3%   +0.10   -0.50
    Tristan

In [10]:
# ========================================
# 📝 EXAMPLE 2: Need specific role (ADC)
# ========================================

my_team = ['Leona', 'Lee Sin', 'Yasuo']  # Support, Jungle, Mid
enemy_team = ['Thresh', 'Graves', 'Ahri', 'Darius']

print("="*60)
print("🎮 DRAFT SITUATION - NEED ADC")
print("="*60)
print(f"\n📘 Your Team: {my_team}")
print(f"📕 Enemy Team: {enemy_team}")
print("\n" + "="*60)
print("🎯 BEST ADC PICKS")
print("="*60 + "\n")

adc_recommendations = recommend_champions(my_team, enemy_team, n_recommendations=10, role_filter='ADC')
print(adc_recommendations.to_string(index=False))

🎮 DRAFT SITUATION - NEED ADC

📘 Your Team: ['Leona', 'Lee Sin', 'Yasuo']
📕 Enemy Team: ['Thresh', 'Graves', 'Ahri', 'Darius']

🎯 BEST ADC PICKS

Champion Role Win % Synergy Counter
   Vayne  ADC 66.8%   +0.55   -0.33
Tristana  ADC 50.9%   +0.33   -0.83
  Ezreal  ADC 48.7%   +0.21   -1.22
    Jinx  ADC 41.4%   +0.00   +0.00
  Kai'Sa  ADC 40.4%   +0.12   -1.45
    Jhin  ADC 39.8%   +0.16   -1.06
  Lucian  ADC 38.4%   -0.07   -1.47
Aphelios  ADC 38.0%   +0.15   -0.63
   Varus  ADC 37.9%   +0.00   -0.25
   Sivir  ADC 37.7%   +0.00   +0.00
Champion Role Win % Synergy Counter
   Vayne  ADC 66.8%   +0.55   -0.33
Tristana  ADC 50.9%   +0.33   -0.83
  Ezreal  ADC 48.7%   +0.21   -1.22
    Jinx  ADC 41.4%   +0.00   +0.00
  Kai'Sa  ADC 40.4%   +0.12   -1.45
    Jhin  ADC 39.8%   +0.16   -1.06
  Lucian  ADC 38.4%   -0.07   -1.47
Aphelios  ADC 38.0%   +0.15   -0.63
   Varus  ADC 37.9%   +0.00   -0.25
   Sivir  ADC 37.7%   +0.00   +0.00


In [11]:
# ========================================
# 📝 EXAMPLE 3: Last pick situation
# ========================================

my_team = ['Ornn', 'Hecarim', 'Syndra', 'Jinx']  # Need support
enemy_team = ['Camille', 'Lee Sin', 'Zed', 'Ezreal', 'Lulu']

print("="*60)
print("🎮 LAST PICK - NEED SUPPORT")
print("="*60)
print(f"\n📘 Your Team: {my_team}")
print(f"📕 Enemy Team: {enemy_team}")
print("\n" + "="*60)
print("🎯 BEST SUPPORT PICKS")
print("="*60 + "\n")

support_recommendations = recommend_champions(my_team, enemy_team, n_recommendations=10, role_filter='SUPPORT')
print(support_recommendations.to_string(index=False))

🎮 LAST PICK - NEED SUPPORT

📘 Your Team: ['Ornn', 'Hecarim', 'Syndra', 'Jinx']
📕 Enemy Team: ['Camille', 'Lee Sin', 'Zed', 'Ezreal', 'Lulu']

🎯 BEST SUPPORT PICKS

Champion    Role Win % Synergy Counter
Nautilus SUPPORT 80.8%   +0.56   -1.79
    Bard SUPPORT 79.8%   +0.38   -1.49
 Alistar SUPPORT 75.4%   +0.21   -1.99
  Thresh SUPPORT 74.9%   +0.12   -1.50
   Janna SUPPORT 73.0%   +0.12   -0.81
    Nami SUPPORT 71.7%   +0.07   -0.33
  Soraka SUPPORT 70.7%   +0.00   +0.00
   Karma SUPPORT 70.6%   +0.00   -0.10
     Lux SUPPORT 70.5%   +0.00   +0.00
    Zyra SUPPORT 70.2%   +0.00   +0.00
Champion    Role Win % Synergy Counter
Nautilus SUPPORT 80.8%   +0.56   -1.79
    Bard SUPPORT 79.8%   +0.38   -1.49
 Alistar SUPPORT 75.4%   +0.21   -1.99
  Thresh SUPPORT 74.9%   +0.12   -1.50
   Janna SUPPORT 73.0%   +0.12   -0.81
    Nami SUPPORT 71.7%   +0.07   -0.33
  Soraka SUPPORT 70.7%   +0.00   +0.00
   Karma SUPPORT 70.6%   +0.00   -0.10
     Lux SUPPORT 70.5%   +0.00   +0.00
    Zyra SUPPORT 

---
## 🕹️ Your Turn - Custom Draft

Modify the cell below with your own draft situation!

In [12]:
# ========================================
# ✏️ YOUR CUSTOM DRAFT
# ========================================

# Enter your team's champions (already picked)
my_team = ['Jinx', 'Thresh']  # <- Modify this!

# Enter enemy team's champions
enemy_team = ['Yasuo', 'Lee Sin']  # <- Modify this!

# Filter by role? Set to None for all roles, or 'TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT'
role_needed = None  # <- Modify this!

# ========================================
print("="*60)
print("🎮 YOUR DRAFT SITUATION")
print("="*60)
print(f"\n📘 Your Team: {my_team if my_team else 'None yet'}")
print(f"📕 Enemy Team: {enemy_team if enemy_team else 'None yet'}")
if role_needed:
    print(f"🎯 Looking for: {role_needed}")
print("\n" + "="*60)
print("🏆 RECOMMENDED PICKS")
print("="*60 + "\n")

my_recommendations = recommend_champions(my_team, enemy_team, n_recommendations=15, role_filter=role_needed)
print(my_recommendations.to_string(index=False))

🎮 YOUR DRAFT SITUATION

📘 Your Team: ['Jinx', 'Thresh']
📕 Enemy Team: ['Yasuo', 'Lee Sin']

🏆 RECOMMENDED PICKS

   Champion    Role Win % Synergy Counter
      Shaco  JUNGLE 89.8%   +0.33   +0.00
      Kayle     TOP 89.7%   +0.39   +0.00
      Riven     TOP 89.6%   +0.30   -0.17
       Ahri     MID 89.4%   +0.30   +0.00
      Senna     ADC 89.3%   +0.30   -0.50
    Rek'Sai  JUNGLE 87.1%   +0.25   -0.50
    Evelynn  JUNGLE 86.8%   +0.25   +0.00
   Xin Zhao  JUNGLE 85.9%   +0.00   +0.00
      Rakan SUPPORT 85.8%   +0.00   -0.50
        Zed     MID 85.3%   +0.21   -0.50
     Lucian     ADC 85.3%   +0.00   -0.44
Mordekaiser     TOP 85.2%   +0.10   +0.00
   Malphite     TOP 85.1%   +0.18   -0.67
      Elise  JUNGLE 85.1%   -0.14   -0.50
       Ekko     MID 85.0%   +0.02   -0.33
   Champion    Role Win % Synergy Counter
      Shaco  JUNGLE 89.8%   +0.33   +0.00
      Kayle     TOP 89.7%   +0.39   +0.00
      Riven     TOP 89.6%   +0.30   -0.17
       Ahri     MID 89.4%   +0.30   +0.00
     

---
## 📊 Draft Analysis Tool

Analyze the full draft to see win probability and team strengths.

In [13]:
def analyze_draft(team_champions, enemy_champions):
    """Analyze a complete or partial draft"""
    
    print("="*70)
    print("📊 DRAFT ANALYSIS")
    print("="*70)
    
    # Teams
    print(f"\n📘 BLUE TEAM: {', '.join(team_champions) if team_champions else 'No picks'}")
    print(f"📕 RED TEAM:  {', '.join(enemy_champions) if enemy_champions else 'No picks'}")
    
    # Win probability
    if team_champions or enemy_champions:
        features = create_features_for_prediction(team_champions, enemy_champions)
        win_prob = best_model.predict_proba(features)[0][1]
        
        print("\n" + "-"*70)
        print(f"\n🎯 PREDICTED WIN RATE: {win_prob*100:.1f}% for Blue Team")
        
        # Visual bar
        bar_length = 50
        blue_bar = int(win_prob * bar_length)
        red_bar = bar_length - blue_bar
        print(f"\n   Blue |{'█' * blue_bar}{'░' * red_bar}| Red")
        print(f"        {win_prob*100:.0f}%{' ' * (bar_length-6)}{(1-win_prob)*100:.0f}%")
    
    # Team synergies
    print("\n" + "-"*70)
    print("\n🤝 SYNERGY ANALYSIS:")
    
    team_synergy = calculate_team_synergy(team_champions)
    enemy_synergy = calculate_team_synergy(enemy_champions)
    
    print(f"   Blue Team Synergy: {team_synergy:+.2f}")
    print(f"   Red Team Synergy:  {enemy_synergy:+.2f}")
    print(f"   Advantage: {'Blue' if team_synergy > enemy_synergy else 'Red'} (+{abs(team_synergy-enemy_synergy):.2f})")
    
    # Best synergy pairs on each team
    if len(team_champions) >= 2:
        best_pair_score = -999
        best_pair = None
        for c1, c2 in combinations(team_champions, 2):
            pair = tuple(sorted([c1, c2]))
            if pair in synergy_data:
                score = synergy_data[pair]
                if isinstance(score, dict):
                    score_val = score.get('synergy_score', 0)
                else:
                    score_val = score
                if score_val > best_pair_score:
                    best_pair_score = score_val
                    best_pair = (c1, c2)
        if best_pair:
            print(f"   Best Blue Duo: {best_pair[0]} + {best_pair[1]} ({best_pair_score:+.2f})")
    
    # Counter analysis
    print("\n" + "-"*70)
    print("\n⚔️ COUNTER ANALYSIS:")
    
    counter_score = calculate_counter_score(team_champions, enemy_champions)
    reverse_counter = calculate_counter_score(enemy_champions, team_champions)
    
    print(f"   Blue vs Red matchups: {counter_score:+.2f}")
    print(f"   Red vs Blue matchups: {reverse_counter:+.2f}")
    
    # Role coverage
    print("\n" + "-"*70)
    print("\n👥 ROLE COVERAGE:")
    
    team_coverage = get_role_coverage(team_champions)
    enemy_coverage = get_role_coverage(enemy_champions)
    
    print(f"   Blue Team: {team_coverage*100:.0f}%")
    print(f"   Red Team:  {enemy_coverage*100:.0f}%")
    
    # Show roles
    print("\n   Blue Roles: ", end="")
    for champ in team_champions:
        print(f"{champ}({get_champion_role(champ)}) ", end="")
    print("\n   Red Roles:  ", end="")
    for champ in enemy_champions:
        print(f"{champ}({get_champion_role(champ)}) ", end="")
    
    print("\n\n" + "="*70)

print("✓ Analysis function ready!")

✓ Analysis function ready!


In [21]:
# ========================================
# 📊 ANALYZE A COMPLETE DRAFT
# ========================================

blue_team = ['Ornn', 'Lee Sin', 'Syndra', 'Jinx', 'Thresh']
red_team = ['Camille', 'Graves', 'Zed', 'Tristana', 'Sona']

analyze_draft(blue_team, red_team)

📊 DRAFT ANALYSIS

📘 BLUE TEAM: Ornn, Lee Sin, Syndra, Jinx, Thresh
📕 RED TEAM:  Camille, Graves, Zed, Tristana, Sona

----------------------------------------------------------------------

🎯 PREDICTED WIN RATE: 52.2% for Blue Team

   Blue |██████████████████████████░░░░░░░░░░░░░░░░░░░░░░░░| Red
        52%                                            48%

----------------------------------------------------------------------

🤝 SYNERGY ANALYSIS:
   Blue Team Synergy: -0.16
   Red Team Synergy:  +0.30
   Advantage: Red (+0.46)
   Best Blue Duo: Jinx + Thresh (+0.25)

----------------------------------------------------------------------

⚔️ COUNTER ANALYSIS:
   Blue vs Red matchups: -3.27
   Red vs Blue matchups: -4.20

----------------------------------------------------------------------

👥 ROLE COVERAGE:
   Blue Team: 100%
   Red Team:  80%

   Blue Roles: Ornn(TOP) Lee Sin(JUNGLE) Syndra(MID) Jinx(ADC) Thresh(SUPPORT) 
   Red Roles:  Camille(TOP) Graves(JUNGLE) Zed(MID) Tristana(ADC

---
## 📋 Available Champions List

Use this to check champion names for your draft!

In [15]:
# Show all available champions by role
print("="*60)
print("📋 AVAILABLE CHAMPIONS BY ROLE")
print("="*60)

for role in ['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT']:
    champs = [c for c in all_champions if c in CHAMPION_ROLES and role in CHAMPION_ROLES[c]]
    print(f"\n{role} ({len(champs)} champions):")
    print(", ".join(sorted(champs)[:20]))
    if len(champs) > 20:
        print(f"   ... and {len(champs)-20} more")

📋 AVAILABLE CHAMPIONS BY ROLE

TOP (40 champions):
Aatrox, Akali, Camille, Cho'Gath, Darius, Dr. Mundo, Fiora, Gangplank, Garen, Gnar, Gragas, Illaoi, Irelia, Jax, Jayce, Kayle, Kennen, Kled, Malphite, Maokai
   ... and 20 more

JUNGLE (35 champions):
Amumu, Diana, Ekko, Elise, Evelynn, Fiddlesticks, Gragas, Graves, Hecarim, Ivern, Jarvan IV, Kayn, Kha'Zix, Kindred, Lee Sin, Lillia, Nidalee, Nocturne, Nunu & Willump, Olaf
   ... and 15 more

MID (37 champions):
Ahri, Akali, Anivia, Annie, Azir, Corki, Diana, Ekko, Fizz, Galio, Irelia, Jayce, Kassadin, Katarina, LeBlanc, Lucian, Lux, Malzahar, Morgana, Orianna
   ... and 17 more

ADC (20 champions):
Aphelios, Ashe, Caitlyn, Draven, Ezreal, Jhin, Jinx, Kai'Sa, Kalista, Kog'Maw, Lucian, Miss Fortune, Samira, Senna, Sivir, Tristana, Twitch, Varus, Vayne, Xayah

SUPPORT (24 champions):
Alistar, Bard, Blitzcrank, Braum, Galio, Janna, Karma, Leona, Lulu, Lux, Maokai, Morgana, Nami, Nautilus, Pantheon, Pyke, Rakan, Senna, Seraphine, Sett
   ..

---
## 🎮 Quick Reference

### How to use:

1. **Get recommendations:**
```python
recommend_champions(['Leona'], ['Thresh', 'Lee Sin'], n_recommendations=10)
```

2. **Filter by role:**
```python
recommend_champions(['Leona', 'Lee Sin'], ['Thresh'], role_filter='ADC')
```

3. **Analyze full draft:**
```python
analyze_draft(['Ornn', 'Lee Sin', 'Syndra', 'Jinx', 'Thresh'],
              ['Camille', 'Graves', 'Zed', 'Ezreal', 'Lulu'])
```

### Tips:
- **Win %** = Predicted win probability if you pick this champion
- **Synergy** = How well this champion works with your current team
- **Counter** = How well this champion does against enemy team

Good luck in your games! 🎮